In [9]:
# from google.colab import drive
# drive.mount("/content/drive/", force_remount=True)

In [10]:
# import zipfile

# with zipfile.ZipFile("./drive/MyDrive/data.zip") as f:
#   f.extractall("")

In [11]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor,Resize,Compose
from torch.utils.data.dataloader import DataLoader

root = "../data/"
BATCH_SIZE = 100

train_ds = ImageFolder(root+"train", Compose([Resize(128),ToTensor()]))
train_dl = DataLoader(train_ds, BATCH_SIZE, shuffle=True, pin_memory=True)

val_ds = ImageFolder(root+"val", Compose([Resize(128),ToTensor()]))
val_dl = DataLoader(val_ds, 49, pin_memory=True)

print(len(train_ds))
img_cls = train_ds.classes

CLS_SIZE = len(img_cls)

print(img_cls)
print(train_ds[0][0].shape)

182000
['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
torch.Size([3, 128, 128])


In [12]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

torch.cuda.empty_cache()

cpu


In [13]:
import torch.nn as nn


class Model(nn.Module):
    def __init__(self):
        super().__init__()

        self.network = nn.Sequential(
            # 3 128
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # 32 64
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            # 64 32
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # 128 16
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            # 128 8
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            # 256 4
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            # 256 2

            nn.Flatten(),
            # 256*2*2
            nn.Linear(256 * 2 * 2, 128),
            nn.ReLU(),
            # 128
            nn.Linear(128, 26)
            # 26
        )

    def forward(self, inputs):

        out = self.network(inputs)
        return torch.softmax(out, dim=-1)

In [14]:
sign_model = Model().to(device)
loss_fn = nn.BCELoss()
opt = torch.optim.Adam(sign_model.parameters(),lr=0.001)

In [15]:
path = "./eyensign/static/sign_model97.pth"
try:
    if torch.cuda.is_available():
        sign_model.load_state_dict(torch.load(path))
        print("Model Loaded(GPU)")
    else:
        sign_model.load_state_dict(torch.load(path,map_location=torch.device("cpu")))
        print("Model Loaded(CPU)")
except Exception as e:
    pass

Model Loaded(CPU)


In [16]:
def fit(epochs):
    acc_max = 0
    for epoch in range(epochs):
        print("Epoch: ",epoch+1)
        # x = 0
        # for images,labels in train_dl:
        #     x += 1
        #     preds = sign_model(images.to(device))

        #     targets = torch.zeros(BATCH_SIZE,CLS_SIZE, dtype=torch.float32)
            
        #     for i in range(BATCH_SIZE):
        #         targets[i,labels[i]]=1

        #     loss = loss_fn(preds.to(device), targets.to(device))
        #     loss.backward()
            
        #     opt.step()
        #     opt.zero_grad()

        #     if x%182 == 0:
        #         print(x,"/",len(train_dl))
        #         print("Loss: ",round(loss.item(),6))
        
        with torch.no_grad():
            correct = 0
            for images,labels in val_dl:
                preds = sign_model(images.to(device))
                for i in range(len(preds)):
                    if (preds[i].max()==preds[i][labels[i].item()]):
                        correct += 1
            
            acc = correct/len(val_ds)
            print("Accuracy: ",round(acc*100,3))
            print("")


fit(1)

Epoch:  1
Accuracy:  97.096

